### pseudo for the priming selector notebook
for each class do 
(
	#Find the 5 best priming pictures
	index[]
	while(len(index) < 5) do (
		#Find the best pictures for priming 
		for each picture NOT IN index[] do
		(
			iCopy = index
			iCopy.append(picture)
			model.train(iCopy)
			history = model.fit(all pictures)
		return index ### of best history acc
		)
	)
	copy picture at index to priming folder
)

In [1]:
monetIndex = [1,2,3,4,5,6,7,8,9]
picassoIndex = []
vangoghIndex = []

In [33]:
from keras import models
from keras.layers import InputLayer, Flatten, Dense, Dropout

def get_model(shape, nClasses):
    return keras.Sequential([
        InputLayer(input_shape=shape),
        Flatten(),    ### Need to flatten if dense NN else if ConvNet no need to flatten.
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(8, activation='relu'),
        Dense(nClasses, activation='sigmoid'),
    ],)

Using TensorFlow backend.


In [24]:
# get the n best pictures for priming, based on nextPicture()
def getPrimingPictures(basePictures, number):
    primingPictures = []
    pictures = basePictures.copy()
    for n in range(number):
        nextP = nextPicture(pictures, primingPictures)
        primingPictures.append(nextP) ## need to append result of the next best performing picture
        pictures.remove(nextP)
    return primingPictures

In [29]:
# find the next picture to include in priming, based on getScore() and bestResult()
def nextPicture(pictures, primingPictures):
    scores = []
    for p in pictures:
        if p not in primingPictures:
            pSet = [p]
            pSet.extend(primingPictures)
            score = getScore(pSet, pictures)
            scores.append({'picture': p, 'score': score})
    #results = getScores(pictures, primingPictures)
    best = bestResult(scores)
    return  best['picture']

In [36]:
# placeholder for defining a score... should be history obejct from model.fit()
def getScore(pSet, pictures):
    return pSet[0] *5

In [39]:
# placeholder for defining a score... should be history obejct from model.fit()
def getScore(pSet, pictures):
    model = get_model((150,150,3), 1)
    model.compile(loss="categorical_crossentropy", optimizer="RMSprop", metrics=["acc"])
    
    history = model.fit(pSet,
        validation_data= pictures,
        epochs= epochs,
        shuffle=True
    )
    return pSet[0] *5

In [31]:
# compare a list of literals with a numeric attribute ['score'], and return the highest.
def bestResult(results):
    best = results[0]
    for r in results:
        if best['score'] < r['score']: best = r
    return best

In [38]:
a = getPrimingPictures(monetIndex, 5)
print(a)

NameError: name 'keras' is not defined

In [40]:
import os
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

keras.__version__

'2.3.1'

In [41]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
# Limits GPU-RAM usage
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [42]:
# import and "clean" data

In [43]:
def get_generator(base_dir, folderName):
    directory = os.path.join(base_dir, folderName)
    return ImageDataGenerator(rescale=1./255).flow_from_directory(
        directory, 
        target_size, 
        batch_size= batch_size,
        shuffle=False
    )

In [64]:
epochs = 50
batch_size = 5
target_width = 150
target_height = 150
target_size = (target_width, target_height)
target_shape = (target_width, target_height, 3) # 3 for color
base_dir = 'shufflePaintings'

In [65]:
train_gen = get_generator(base_dir, 'train')

#gen = get_generator(base_dir, 'train')
#monet_gen = get_images(gen)
#picasso_gen = get_generator(base_dir, 'train\picasso')
#vangogh_gen = get_generator(base_dir, 'train\vangogh')

Found 210 images belonging to 3 classes.


In [66]:
numOfClasses = train_gen.num_classes

#numOfPrimingSamples = priming_gen.n
numOfTrainSamples = train_gen.n
#numOfTestSamples = test_gen.n
#numOfValSamples = validation_gen.n

In [67]:
# Define Conv model

In [68]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape= target_shape) # honestly, I'm not sure whether height or width comes first

In [69]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [70]:
block3_pool_width = 18
block3_pool_height = 18
block3_pool_area = block3_pool_width * block3_pool_height
block3_pool_no_of_filters = 256

## Following not used, but included for completeness and your own possible experiments
block5_pool_width = 4
block5_pool_height = 4
block5_pool_area = block3_pool_width * block3_pool_height
block5_pool_no_of_filters = 512

In [71]:
from keras import Model

intermediate_layer_model = Model(inputs=conv_base.input, 
                                 outputs=conv_base.get_layer('block3_pool').output)
no_of_filters_left = 100

In [72]:
from keras.utils.np_utils import to_categorical

def extract_features(generator, sample_count):
    features = np.zeros(shape=(sample_count, block3_pool_width, block3_pool_height, block3_pool_no_of_filters))
    labels = np.zeros(shape=(sample_count))
    
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = intermediate_layer_model.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break

    ### Now throwing away most of the features (= results of features)
    features = features[:,:,:,0:no_of_filters_left]
    ### convert labels 0, 1, and 2 to [1,0,0], [0,1,0] and [0,0,1] 
    labels = to_categorical(labels)
    return features, labels

In [73]:
import numpy as np

train_features, train_labels = extract_features(train_gen, numOfTrainSamples)

#validation_features, validation_labels = extract_features(validation_gen, numOfValSamples)

#test_features, test_labels = extract_features(test_gen, numOfTestSamples)

ValueError: could not broadcast input array from shape (5,3) into shape (5)

In [ ]:
train_features = np.reshape(train_features, (numOfTrainSamples, block3_pool_area * no_of_filters_left))
validation_features = np.reshape(validation_features, (numOfValSamples, block3_pool_area * no_of_filters_left))
test_features = np.reshape(test_features, (numOfTestSamples, block3_pool_area * no_of_filters_left))

In [ ]:
# Define Dense model

In [ ]:
import keras
from keras import models
from keras import layers

def get_model(shape):
    model = models.Sequential()
    model.add(layers.Dense(16, input_dim= shape))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
my_model = get_model(1)

In [ ]:
my_model.summary()


In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, input_dim= block3_pool_area * no_of_filters_left))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(numOfClasses, activation='sigmoid'))

In [ ]:
# train model

In [ ]:
from keras import optimizers

model.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(lr=2e-5), metrics=["acc"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_features, 
    train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(validation_features, validation_labels))

In [ ]:
#model.save('baseline.h5')

In [ ]:
# validation

In [ ]:
import matplotlib.pyplot as plt

def graph(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(len(acc))
    
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    
    plt.show()
    

In [ ]:
graph(history)